## 2.3. feature engeineering categoricals and dealing with missing values

In [307]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [308]:
data_raw = pd.read_csv('perenials_dataset.csv')
data_raw.head()

,Unnamed: 0,id,idstudy,plotid,country,region,temp_c,precipitation_mm,climate,latitud,longitud,bedrock,soil_type,usda,type_change,crop_current,crop_type,cropage,current_land_use,luc_this_study,previous_land_use,years_since_luc,temporal,timetemporal,change_management,year_since_management_change,soil_treatments,tillageyesno,tillage_type,fertilizeryesno,fet_type,rate_fert,pesticides,org_amendyesno,org_amend_type,prev_man,multiple_sampl,dif_depth_current_point,conti_depth,num_depth,year_measure,n_plots_current,n_sample_per_plot_current,n_measured_current,soil_from_cm_current,soil_to_cm_current,depth_midpoint_current,bulk_density_current,%clay_current,%silt_current,%sand_current,ph_current,soc_g_kg_current,sd_soc_gkg_current,soc_mg_ha_current,sd_soc_mgha_current,n_plots_previous,n_sample_per_plot_previous,n_measured_current.1,soil_from_cm_previous,soil_to_cm_previous,depth_midpoint_previous,bulkdensity_previous,%clay_previous,%silt_previous,%sand_previous,ph_previous,soc_g_kg_previous,sd_soc_gkg_previous,soc_mg_ha_previous,sd_soc_mgha_previous,yearppub
0,0,1,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,0.0,10.0,5.0,1.30,13.7,5.1,81.2,NaN,NaN,NaN,20.70,NaN,20.0,1.0,20,0.0,10.0,5.0,1.36,19.2,10.4,70.5,NaN,NaN,NaN,17.39,NaN,2017.0
1,1,2,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,10.0,20.0,15.0,1.34,14.8,1.1,84.2,NaN,NaN,NaN,12.54,NaN,20.0,1.0,20,10.0,20.0,15.0,1.58,21.7,8.9,69.3,NaN,NaN,NaN,15.57,NaN,2017.0
2,2,3,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,20.0,60.0,40.0,1.55,18.0,2.0,80.0,NaN,NaN,NaN,30.92,NaN,20.0,1.0,20,20.0,60.0,40.0,1.41,24.8,8.7,66.5,NaN,NaN,NaN,55.38,NaN,2017.0
3,3,4,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,60.0,100.0,80.0,1.46,18.8,3.5,77.7,NaN,NaN,NaN,36.23,NaN,20.0,1.0,20,60.0,100.0,80.0,1.31,24.8,9.2,66.0,NaN,NaN,NaN,36.10,NaN,2017.0
4,4,5,1,2,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Luvisol,Ultisol,LUC,Sugarcane,grass,4.0,bioenergy,YES,perennial_food,4.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,0.0,10.0,5.0,1.20,38.1,10.9,51.0,NaN,NaN,NaN,16.43,NaN,20.0,1.0,20,0.0,10.0,5.0,1.27,42.9,11.4,45.7,NaN,NaN,NaN,16.28,NaN,2017.0


### 2.3.1 Feature no till
This feature shows a lot of missing values. However, with the variables previous crop, current crop and, land_use_change (luc) and soil treatments me are able to generate a nice feature here.

In [309]:
data.soil_treatments.value_counts()

NO             260
YES            148
grazed          18
plow/harrow     12
disk_harrow      7
tillage          4
burn_5years      2
burn_annual      2
Name: soil_treatments, dtype: int64

In [310]:
data = data_raw.copy()


In [311]:
data.previous_land_use.value_counts()

annual_crop         441
grassland           324
natural_forest      232
secondary_forest     62
SRC                  21
fallow               18
perennial_food        8
Name: previous_land_use, dtype: int64

In [312]:
data.current_land_use.value_counts()

bioenergy        612
food             599
SRC              296
bioproduct        37
livestockfood     25
agrofor           18
annual_crop       18
Name: current_land_use, dtype: int64

In [313]:
data_till = data_raw[['id']]
data_till['soil_treatments'] = data['soil_treatments']

In [314]:
data_till['curr_till'] = data_raw['crop_current'].replace({"Sugarcane" : 1, 
                                                                'vineyard':1, 
                                                                "Cotton?corn?groundnut" : 1, 
                                                                'Maize' : 1, 
                                                                'Cucumber': 1, 
                                                                'Melon': 1 })
data_till['curr_till'] = data_till['curr_till'].apply(lambda x: 0 if x!=1 else x)
data_till['curr_till'].value_counts()                                                  

0    1354
1     251
Name: curr_till, dtype: int64

In [315]:
data_till['curr_till'] = np.where((data_till['soil_treatments'] == 'NO') |
                                  (data_till['soil_treatments'] == 'grazed'),0, data_till['curr_till'])     
data_till['curr_till'] = np.where((data_till['soil_treatments'] == 'YES') | 
                                  (data_till['soil_treatments'] == 'plow/harrow') |
                                  (data_till['soil_treatments'] == 'disc/harrow') |
                                  (data_till['soil_treatments'] == 'tillage')
                                  ,1, data_till['curr_till']) 
data_till.curr_till.value_counts()

0    1211
1     394
Name: curr_till, dtype: int64

In [316]:
data_till['soil_treatments'] = data['soil_treatments']
data_till.head()

,id,soil_treatments,curr_till
0,1,NaN,1
1,2,NaN,1
2,3,NaN,1
3,4,NaN,1
4,5,NaN,1


In [317]:
data_till['prev_till'] = data_raw['previous_land_use'].replace({"annual_crop" : 1, 
                                                                'grassland' : 0, 
                                                                'natural_forest': 0, 
                                                                'secondary_forest': 0, 
                                                                'SRC': 1, 'fallow':1, 
                                                                "perennial_food" : 0})

data_till.prev_till.value_counts()

0.0    626
1.0    480
Name: prev_till, dtype: int64

Feature generation till
as we want to predict the change in SOC we are onlu interested in the chance in the tilling practice. Thereofore we assume we have only three cases
0 no change  
-1 from no till to till (we expect a negative effect on SOC)  
1 from till to no till (we expect a positive effect on SOC)

In [318]:
data_till['change_till'] = np.nan
data_till['change_till'] = np.where((data_till['prev_till'] == 0) & (data_till['curr_till'] == 0),0, data_till['change_till'])
data_till['change_till'] = np.where((data_till['prev_till'] == 1) & (data_till['curr_till'] == 1),0, data_till['change_till'])
data_till['change_till'] = np.where((data_till['prev_till'] == 0) & (data_till['curr_till'] == 1),-1, data_till['change_till'])
data_till['change_till'] = np.where((data_till['prev_till'] == 1) & (data_till['curr_till'] == 0),1, data_till['change_till'])
data_till.head()

,id,soil_treatments,curr_till,prev_till,change_till
0,1,NaN,1,0.0,-1.0
1,2,NaN,1,0.0,-1.0
2,3,NaN,1,0.0,-1.0
3,4,NaN,1,0.0,-1.0
4,5,NaN,1,0.0,-1.0


In [319]:
data_till['change_till'].value_counts()

 0.0    446
 1.0    426
-1.0    234
Name: change_till, dtype: int64

In [320]:
data_till.drop(['soil_treatments', 'curr_till', 'prev_till' ], axis=1, inplace = True)

### 2.3.2 feature fertilzer

In [321]:
data_till['fert_type'] = data['fet_type']

In [322]:

data_till['fert_type'].replace(to_replace=['N', 'NPK', 'NO_N', 'reduced_N', 'high_N', 'less_N', 'more_N', 'NP', 'N?P'], value ='NO' , inplace=True)
data_till['fert_type'].replace(to_replace=['urea', 'waste', 'organic?N', 'less_vinasse', 'more_vinasse', 'less_trash', 'manure_+_CC', 'more_trash', 'manure', 'DAP+urea'], value ='organic' , inplace=True)
data_till['fert_type'].replace(to_replace=['biochar15%', 'biochar30%'], value ='biochar' , inplace=True)
data_till['fert_type'].replace(to_replace=['PK+(NH4)2SO4',
'PK+(NH4)2SO11',
'PK+(NH4)2SO19',
'PK+(NH4)2SO7',
'PK+(NH4)2SO12',
'PK+(NH4)2SO8',
'PK+(NH4)2SO16',
'PK+(NH4)2SO20',
'PK+(NH4)2SO21',
'PK+(NH4)2SO15',
'PK+NH4NO3',
'PK+(NH4)2SO10',
'PK+(NH4)2SO17',
'PK+(NH4)2SO18',
'PK+NH4NO4', 
'PK+NH4NO5',
'PK+(NH4)2SO14',
'PK+(NH4)2SO9','PK+(NH4)2SO13', 'PK'], value ='mineral' , inplace=True)
data_till['fert_type'].replace(to_replace=['mineral_changing_to_organic',
'DNPK',
'DNP',
'OMW_+_CR'], value ='mixed' , inplace=True)

In [323]:
data_till['fert_type'].value_counts()

NO         183
mineral    120
organic    119
mixed       75
compost     11
biochar      6
Name: fert_type, dtype: int64

In [324]:
data_till.dtypes

id               int64
change_till    float64
fert_type       object
dtype: object

In [326]:
data_till.to_csv('df_cats.csv')